In [1]:
import pandas as pd
from random import randint
import re

import utils

# Emotion Data Prep

In [3]:
edf = pd.read_csv('data/emotions/train.txt', sep=';', header=0, names=['text', 'emotion'])
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     15999 non-null  object
 1   emotion  15999 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


## Optional: Deal with spellings like "don t"

In [9]:
# Here if we want to use it. Leaving it off for now.
def remove_contracted_terms(raw_text: str) -> str:
    sep_patterns = [
        r"[\s][^\s]+n[\s]+t[\s]",
        r"[\s][Ii][\s]+m[\s]",
        r"you[\s]+re[\s]",
        r"they[\s]+re[\s]",
        r"she[\s]+s[\s]",
        r"[\s]he[\s]+s[\s]",
        r"[\s][^\s]+[\s]+d[\s]",
        r"[\s][^\s]+[\s]+ve[\s]"
    ]
    combo_pattern = re.compile('|'.join(sep_patterns))
    return re.sub(combo_pattern, ' ', raw_text)


In [10]:
edf['contractions_removed'] = edf['text'].apply(remove_contracted_terms)
edf.sample(n=5)

,text,emotion,contractions_removed
7680,i dont recall just now yet vividly recall look...,love,i dont recall just now yet vividly recall look...
5747,i feel so embarrassed of myself for even havin...,sadness,i feel so embarrassed of myself for even havin...
4466,i would feel so excited waiting for the mailma...,joy,i would feel so excited waiting for the mailma...
14576,i growled at her i began to feel extremely ann...,anger,i growled at her i began to feel extremely ann...
4460,i am right now i feel amused the sounds i hear...,joy,i am right now i feel amused the sounds i hear...


In [14]:
edf.sample(n=5)

,text,emotion,contractions_removed
4198,i feel that learning more about animals and th...,joy,i feel that learning more about animals and th...
9552,i am feeling a bit apprehensive about carrying...,fear,i am feeling a bit apprehensive about carrying...
7054,im feeling very bitter against knight in shini...,anger,im feeling very bitter against knight in shini...
10405,i don t always remember to do this but when i ...,anger,i always remember to do this but when feeling ...
6786,i feel shes friendly and nice,joy,i feel shes friendly and nice


In [15]:
edf.drop(columns=['contractions_removed'], inplace=True)
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15999 entries, 0 to 15998
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   text     15999 non-null  object
 1   emotion  15999 non-null  object
dtypes: object(2)
memory usage: 250.1+ KB


## Binary Label: Negative Emotion ("Stress") as 1

In [3]:
edf['emotion'].value_counts()

emotion
joy         5362
sadness     4665
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

Just going to roughly group "sadness", "anger", and "fear" into "stressed"

In [4]:
possible_stress = {'sadness', 'anger', 'fear'}
edf['label'] = edf['emotion'].apply(lambda emotion: 1 if emotion.strip() in possible_stress else 0)
edf.sample(n=3)

,text,emotion,label
12897,i dont have a god to turn to doesnt mean i don...,anger,1
6288,i wasnt feeling when i got on board but its re...,joy,0
2660,im totally walking on sunshine feeling lighter...,sadness,1


## Text Processing

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from urllib.parse import urlparse

In [6]:
lemmatizer = WordNetLemmatizer()
stop_words = list(stopwords.words('english'))

In [7]:
print(len(stop_words))
utils.add_stopwords_missing_apostrophe(stop_words)
print(len(stop_words))

179
205


In [8]:
edf['processed_text'] = edf['text'].apply(lambda raw_text: utils.process_text(
    text_chunk=raw_text, stopwords=stop_words, lemmatizer_obj=lemmatizer
))
edf.sample(n=5)

,text,emotion,label,processed_text
13569,ill feel even more pressured,fear,1,ill feel even pressured
7671,i look at his sweet little face crying for his...,sadness,1,look sweet little face cry mama wanting hold l...
12940,i was in the firm i feel passionate about what...,love,0,firm feel passionate challenge mind intellect ...
2815,i look forward to when i am feeling better and...,joy,0,look forward feeling better write often
11223,i feel so virtuous,joy,0,feel virtuous


## Vectorize (Word Embedding)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

### Experiment with Max and Min Document Frequencies for a Reasonable Vocabulary Size

In [24]:
doc_freq_maxes = [0.9, 0.8, 0.7, 0.6, 0.5]

In [27]:
for ceiling in doc_freq_maxes:
    tfidf = TfidfVectorizer(max_df=ceiling)
    tf_df = tfidf.fit_transform(edf['processed_text'])
    tf_df.toarray()
    print(f'Max doc freq: {ceiling}\nTerms: {tf_df.shape[1]}\n')

Max doc freq: 0.9
Terms: 13435

Max doc freq: 0.8
Terms: 13435

Max doc freq: 0.7
Terms: 13435

Max doc freq: 0.6
Terms: 13434

Max doc freq: 0.5
Terms: 13434



In [32]:
doc_freq_mins = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.]

In [33]:
for floor in doc_freq_mins:
    tfidf = TfidfVectorizer(min_df=floor)
    tf_df = tfidf.fit_transform(edf['processed_text'])
    tf_df.toarray()
    print(f'Min doc freq: {floor}\nTerms: {tf_df.shape[1]}\n')

Min doc freq: 0.1
Terms: 4

Min doc freq: 0.01
Terms: 101

Min doc freq: 0.001
Terms: 1363

Min doc freq: 0.0001
Terms: 6496

Min doc freq: 1e-05
Terms: 13435

Min doc freq: 0.0
Terms: 13435



### Actual Embedding

In [10]:
# Try max_df = 0.6 and min_df = 0.0001
MAX_DF = 0.6
MIN_DF = 0.0001

In [11]:
tfidf = TfidfVectorizer(min_df=MIN_DF, max_df=MAX_DF)
tf_df = tfidf.fit_transform(edf['processed_text'])
tf_df.toarray()
tf_df.shape

(15999, 6495)

In [12]:
tf_df = pd.DataFrame(tf_df.toarray(), columns=tfidf.get_feature_names_out())
tf_df.sample(n=3)

,aa,abandon,abandoned,abandoning,abandonment,abc,abdomen,abide,ability,abit,...,zach,zealand,zen,zero,zest,zombie,zone,zoom,zooming,zumba
8925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
tf_df.describe()

,aa,abandon,abandoned,abandoning,abandonment,abc,abdomen,abide,ability,abit,...,zach,zealand,zen,zero,zest,zombie,zone,zoom,zooming,zumba
count,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,...,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000,15999.000000
mean,0.000088,0.000080,0.000250,0.000076,0.000092,0.000111,0.000112,0.000084,0.000784,0.000138,...,0.000072,0.000096,0.000053,0.000137,0.000049,0.000127,0.000239,0.000051,0.000054,0.000067
std,0.006486,0.006044,0.012614,0.006844,0.006779,0.007231,0.007563,0.006296,0.016766,0.009148,...,0.006537,0.006198,0.004726,0.007408,0.004508,0.007301,0.009740,0.004746,0.005011,0.004937
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.521035,0.581781,0.818975,0.667780,0.586314,0.610155,0.729999,0.594738,0.541364,0.771920,...,0.676502,0.491465,0.430916,0.594343,0.488050,0.492643,0.513285,0.531097,0.552705,0.384099


In [41]:
# Either run this cell or the next, NOT both - this is for reproducible shuffling of train/test data
random_seed = randint(0, 50)
random_seed

5

In [13]:
# Either run this cell or the above, NOT both
random_seed = 5

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    tf_df, edf['label'], test_size=0.2, random_state=random_seed, stratify=edf['label']
)
X_train.shape, y_train.shape

((12799, 6495), (12799,))

In [18]:
X_test.shape, y_test.shape

((3200, 6495), (3200,))

# ML Models

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

## Logistic Regression

In [46]:
lr_model = LogisticRegression().fit(X_train, y_train)
lr_model.score(X_test, y_test)

0.95375

## Naive Bayes

In [47]:
nb_model = MultinomialNB().fit(X_train, y_train)
nb_model.score(X_test, y_test)

0.933125

## Random Forest

In [48]:
rf_model = RandomForestClassifier().fit(X_train, y_train)
rf_model.score(X_test, y_test)

0.940625

# DL Models

In [14]:
import tensorflow as tf

2024-03-05 19:29:42.253353: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-05 19:29:42.473517: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-05 19:29:42.473576: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-05 19:29:42.474379: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-05 19:29:42.557664: I tensorflow/core/platform/cpu_feature_g

## RNN

Adapted from [TensorFlow Text Classification RNN Tutorial](https://www.tensorflow.org/text/tutorials/text_classification_rnn)

In [20]:
DIM_NUM = 64

In [22]:
rnn_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(
        input_dim=X_train.shape[1],
        output_dim=DIM_NUM,
        mask_zero=True), # Masking to handel variable sequence lengths? Is this even necessary?
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(DIM_NUM)),
    tf.keras.layers.Dense(DIM_NUM, activation='relu'),
    tf.keras.layers.Dense(1)
])

2024-03-05 19:45:15.602837: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-03-05 19:45:15.602898: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: rainbow
2024-03-05 19:45:15.602907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: rainbow
2024-03-05 19:45:15.603113: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.161.7
2024-03-05 19:45:15.603138: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 535.154.5
2024-03-05 19:45:15.603144: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 535.154.5 does not match DSO version 535.161.7 -- cannot find working devices in this configuration


## LSTM